# Get Erowid data
<br>
James Chapman<br>
CIS 830 Advanced Topics in AI – Term Project<br>
Kansas State University<br><br>

This notebook collects URLs from Erowid's [Main Chart of substances](https://www.erowid.org/general/big_chart.shtml).<br>

This will be used for RAG (the text/HTML/content from these URLs will be ingested by langchain)<br>


#### Main Chart of substances <br>

Each substance has a category and name (often slightly different than other websites).<br>

columns (each with a URL)
- MainPage  
- Basics  
- Images  
- Law  
- Dose  
- Experiences  
- FAQ  
- Effects  
- Chemistry  
- Testing  
- Health  
- History  
- Spiritual  
- Cultivation  
- Books  
- Journals  
- Writings  
- Media  


### Saves CSV with the following columns (for each URL)

- drug
- substance
- category
- erowid_column
- url  

In [1]:
import requests
import pandas as pd
import requests

from urllib.parse import urljoin
from bs4 import BeautifulSoup
from fuzzywuzzy import process
from tqdm import tqdm
tqdm.pandas()

In [2]:

CHART_URL = "https://www.erowid.org/general/big_chart.shtml"
headers    = {"User-Agent": "Mozilla/5.0"}

html   = requests.get(CHART_URL, headers=headers, verify=False).text
soup   = BeautifulSoup(html, "lxml")

# ## helpers ##########################################
def make_absolute(href: str | None) -> str | None:
    """prepend site root to relative links; leave None / empty untouched"""
    return urljoin(CHART_URL, href) if href else None

# ## extract ##########################################-
records           : list[dict]  = []
columns_by_table  : dict[str, list[str]] = {}

for table in soup.select("table.big-chart-surround"):
    category = table.get("id", "UNKNOWN").removeprefix("section-").upper()

    # header row:  <td>Category name</td> followed by 18 <th> containing <img alt="">
    header_ths = table.select("tr:first-child th")
    col_names  = [img["alt"] if (img := th.find("img")) and img.get("alt") else f"Col_{i}"
                  for i, th in enumerate(header_ths)]
    columns_by_table[category] = col_names

    # data rows
    for tr in table.select("tr")[1:]:
        cells        = tr.find_all(["td", "th"])
        first_cell   = cells[0]
        substance    = first_cell.get_text(strip=True)
        substance_url = make_absolute(first_cell.a["href"]) if first_cell.a else None

        row = {
            "Substance": substance,
            "URL"      : substance_url,
            "Category" : category,
        }

        # remaining cells map 1‑to‑1 onto col_names
        for col_name, th in zip(col_names, cells[1:]):
            row[col_name] = make_absolute(th.a["href"]) if th and th.a else None

        records.append(row)

# ## build DataFrame ####################################
erowid_data = pd.DataFrame(records).sort_values(["Category", "Substance"]).reset_index(drop=True)

# ## verify column consistency ##############################
first_cols = None
inconsistent = False
for cat, cols in columns_by_table.items():
    if first_cols is None:
        first_cols = cols
    elif cols != first_cols:
        inconsistent = True
        print(f"[WARN] Column mismatch in “{cat}”:\n       {cols}")

if not inconsistent:
    print("All category tables share identical columns.")

erowid_data.info(verbose=True)
erowid_data


c:\Users\James\anaconda3\envs\SparKG\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.erowid.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


All category tables share identical columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Substance    362 non-null    object
 1   URL          349 non-null    object
 2   Category     362 non-null    object
 3   MainPage     349 non-null    object
 4   Basics       78 non-null     object
 5   Images       305 non-null    object
 6   Law          287 non-null    object
 7   Dose         88 non-null     object
 8   Experiences  20 non-null     object
 9   FAQ          25 non-null     object
 10  Effects      71 non-null     object
 11  Chemistry    120 non-null    object
 12  Testing      51 non-null     object
 13  Health       26 non-null     object
 14  History      67 non-null     object
 15  Spiritual    1 non-null      object
 16  Cultivation  16 non-null     object
 17  Books        0 non-null      object
 18  Journals     66 non-null 

,Substance,URL,Category,MainPage,Basics,Images,Law,Dose,Experiences,FAQ,...,Chemistry,Testing,Health,History,Spiritual,Cultivation,Books,Journals,Writings,Media
0,bee,https://www.erowid.org/animals/bee/bee.shtml,ANIMALS,https://www.erowid.org/animals/bee/bee.shtml,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,blackwidow,https://www.erowid.org/animals/blackwidow/blac...,ANIMALS,https://www.erowid.org/animals/blackwidow/blac...,None,https://www.erowid.org/animals/blackwidow/blac...,None,None,https://www.erowid.org/experiences/subs/exp_An...,None,...,None,None,None,None,None,None,None,None,None,None
2,frogs,https://www.erowid.org/animals/frogs/frogs.shtml,ANIMALS,https://www.erowid.org/animals/frogs/frogs.shtml,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,phyllomedusa,https://www.erowid.org/animals/phyllomedusa/ph...,ANIMALS,https://www.erowid.org/animals/phyllomedusa/ph...,None,https://www.erowid.org/animals/phyllomedusa/ph...,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,toads,https://www.erowid.org/animals/toads/toads.shtml,ANIMALS,https://www.erowid.org/animals/toads/toads.shtml,None,https://www.erowid.org/animals/toads/toads_ima...,https://www.erowid.org/animals/toads/toads_law...,None,None,None,...,None,None,None,None,None,None,None,https://www.erowid.org/animals/toads/toads_jou...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357,selegiline,https://www.erowid.org/smarts/selegiline/seleg...,SMARTS,https://www.erowid.org/smarts/selegiline/seleg...,https://www.erowid.org/smarts/selegiline/seleg...,https://www.erowid.org/smarts/selegiline/seleg...,https://www.erowid.org/smarts/selegiline/seleg...,None,None,None,...,https://www.erowid.org/smarts/selegiline/seleg...,None,None,None,None,None,None,https://www.erowid.org/smarts/selegiline/seleg...,None,None
358,tryptophan,https://www.erowid.org/smarts/tryptophan/trypt...,SMARTS,https://www.erowid.org/smarts/tryptophan/trypt...,https://www.erowid.org/smarts/tryptophan/trypt...,https://www.erowid.org/smarts/tryptophan/trypt...,https://www.erowid.org/smarts/tryptophan/trypt...,None,https://www.erowid.org/experiences/subs/exp_Tr...,None,...,https://www.erowid.org/smarts/tryptophan/trypt...,None,None,None,None,None,None,None,None,None
359,vasopressin,https://www.erowid.org/smarts/vasopressin/vaso...,SMARTS,https://www.erowid.org/smarts/vasopressin/vaso...,https://www.erowid.org/smarts/vasopressin/vaso...,https://www.erowid.org/smarts/vasopressin/vaso...,https://www.erowid.org/smarts/vasopressin/vaso...,None,https://www.erowid.org/experiences/subs/exp_Va...,None,...,https://www.erowid.org/smarts/vasopressin/vaso...,None,None,None,None,None,None,https://www.erowid.org/smarts/vasopressin/vaso...,None,None
360,vinpocetine,https://www.erowid.org/smarts/vinpocetine/vinp...,SMARTS,https://www.erowid.org/smarts/vinpocetine/vinp...,None,https://www.erowid.org/smarts/vinpocetine/vinp...,None,None,None,None,...,https://www.erowid.org/smarts/vinpocetine/vinp...,None,None,None,None,None,None,None,None,None


In [3]:
# ################################################--
# 1. compare “MainPage” vs. substance URL
# ################################################--
erowid_data["Main_equals_URL"] = erowid_data["MainPage"].fillna("") == erowid_data["URL"].fillna("")

mismatched = erowid_data.loc[~erowid_data["Main_equals_URL"], 
                    ["Category", "Substance", "URL", "MainPage"]]

if mismatched.empty:
    print("Every row has identical URL and MainPage values.")
else:
    print(f"Found {len(mismatched)} rows where they differ:")
    print(mismatched.to_string(index=False))


Every row has identical URL and MainPage values.


In [4]:
erowid_data = erowid_data.drop(['URL','Main_equals_URL','Images','Books'], axis=1)
erowid_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Substance    362 non-null    object
 1   Category     362 non-null    object
 2   MainPage     349 non-null    object
 3   Basics       78 non-null     object
 4   Law          287 non-null    object
 5   Dose         88 non-null     object
 6   Experiences  20 non-null     object
 7   FAQ          25 non-null     object
 8   Effects      71 non-null     object
 9   Chemistry    120 non-null    object
 10  Testing      51 non-null     object
 11  Health       26 non-null     object
 12  History      67 non-null     object
 13  Spiritual    1 non-null      object
 14  Cultivation  16 non-null     object
 15  Journals     66 non-null     object
 16  Writings     11 non-null     object
 17  Media        19 non-null     object
dtypes: object(18)
memory usage: 51.0+ KB


In [5]:
substances_of_interest = pd.read_csv('../data/substances_of_interest_list.csv', encoding="utf-8-sig")
substances_of_interest

,drug
0,"2,6-Xylidine"
1,2-amino-5-chloropyridine
2,2-fluoro-2-oxo PCE
3,2-Oxo-3-hydroxy-LSD
4,3-hydroxy flubromazepam
...,...
72,Speciociliatine
73,Temazepam
74,Xylazine
75,Zolpidem


In [6]:
matching_erowid_data = substances_of_interest.copy()
for col in erowid_data.columns:
    if col not in matching_erowid_data.columns:
        matching_erowid_data[col] = None          

for index, row in matching_erowid_data.iterrows():
    drug = row["drug"]  

    # Find the closest match in the matching_erowid_data DataFrame
    match, score = process.extractOne(drug, erowid_data["Substance"].tolist())
    if score > 98:  ############################### 
        src_row = erowid_data.loc[erowid_data["Substance"] == match].iloc[0]

        # copy every column’s value into the target DataFrame
        for col in erowid_data.columns:
            matching_erowid_data.at[index, col] = src_row[col]
        print(match)


buprenorphine
codeine
cyclobenzaprine
diphenhydramine
fentanyl
gabapentin
hydrocodone
hydromorphone
ketamine
lorazepam
lsd
mda
mdma
meperidine
methadone
morphine
naltrexone
oxycodone
oxymorphone
pregabalin
psilocybin
temazepam
zolpidem
zopiclone


In [7]:
for index, row in matching_erowid_data.iterrows():
    if not pd.isna(row["Substance"]):
        continue
    drug = row["drug"]  
    # Find the top three closest matches 
    top_matches = process.extract(drug, erowid_data["Substance"].tolist(), limit=3)
    print("########################################")
    for match, score in top_matches:
        if score > 50:  ################################## 
            print(f"{drug} | {match} | {score}")

#I skimmed through these to find errors

########################################
2,6-Xylidine | doi | 60
2,6-Xylidine | lsd | 60
2,6-Xylidine | clonidine | 57
########################################
2-amino-5-chloropyridine | 5-mapb | 86
2-amino-5-chloropyridine | mint | 68
2-amino-5-chloropyridine | chlorpromazine | 64
########################################
2-fluoro-2-oxo PCE | 2ce | 60
2-fluoro-2-oxo PCE | pcp | 60
########################################
2-Oxo-3-hydroxy-LSD | lsd | 90
2-Oxo-3-hydroxy-LSD | 1p-lsd | 86
2-Oxo-3-hydroxy-LSD | 3-meo-pcp | 86
########################################
3-hydroxy flubromazepam | bromazepam | 90
3-hydroxy flubromazepam | 3-meo-pcp | 86
3-hydroxy flubromazepam | amt | 72
########################################
3-hydroxy flubromazepam glucuronide | bromazepam | 90
3-hydroxy flubromazepam glucuronide | 3-meo-pcp | 86
3-hydroxy flubromazepam glucuronide | lorazepam | 70
########################################
4-ANPP | pcp | 72
4-ANPP | a-pvp | 55
4-ANPP | mptp | 51
###############

In [8]:
# the fuzzy matching missed 2 matches
# Amphetamine and Methamphetamine
def patch_row(target_df, drug_name, erowid_name):
    # full source Series 
    src_row = erowid_data.loc[erowid_data["Substance"] == erowid_name].iloc[0]
    # write every column into the matching row of the target DataFrame
    for col in erowid_data.columns:
        target_df.loc[target_df["drug"] == drug_name, col] = src_row[col]

patch_row(matching_erowid_data, "Amphetamine", "amphetamines")
patch_row(matching_erowid_data, "Methamphetamine", "meth")

matching_erowid_data.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   drug         77 non-null     object
 1   Substance    26 non-null     object
 2   Category     26 non-null     object
 3   MainPage     26 non-null     object
 4   Basics       6 non-null      object
 5   Law          25 non-null     object
 6   Dose         10 non-null     object
 7   Experiences  0 non-null      object
 8   FAQ          5 non-null      object
 9   Effects      9 non-null      object
 10  Chemistry    9 non-null      object
 11  Testing      11 non-null     object
 12  Health       6 non-null      object
 13  History      9 non-null      object
 14  Spiritual    0 non-null      object
 15  Cultivation  0 non-null      object
 16  Journals     5 non-null      object
 17  Writings     4 non-null      object
 18  Media        4 non-null      object
dtypes: object(19)
memory usage: 11.

In [9]:

url_cols = [c for c in matching_erowid_data.columns
            if c not in ("drug", "Substance", "Category")]

sets = []
for _, row in matching_erowid_data.iterrows():
    drug = row["drug"]
    substance = row["Substance"]
    category = row["Category"]
    for col in url_cols:
        url = row[col]
        if isinstance(url, str) and url.strip():        # non‑empty link
            sets.append((drug, substance, category, col, url.strip()))

print(f"{len(sets)} (drug, substance, category, col, url) sets")

long_matching_erowid_data = pd.DataFrame(sets,
    columns=["drug","substance","category","erowid_column","url"]
)   

long_matching_erowid_data.info(verbose=True)
long_matching_erowid_data.to_csv('../data/long_matching_erowid_data.csv', index=False, encoding="utf-8-sig")
long_matching_erowid_data


129 (drug, substance, category, col, url) sets
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   drug           129 non-null    object
 1   substance      129 non-null    object
 2   category       129 non-null    object
 3   erowid_column  129 non-null    object
 4   url            129 non-null    object
dtypes: object(5)
memory usage: 5.2+ KB


,drug,substance,category,erowid_column,url
0,Amphetamine,amphetamines,CHEMICALS,MainPage,https://www.erowid.org/chemicals/amphetamines/...
1,Amphetamine,amphetamines,CHEMICALS,Basics,https://www.erowid.org/chemicals/amphetamines/...
2,Amphetamine,amphetamines,CHEMICALS,Law,https://www.erowid.org/chemicals/amphetamines/...
3,Amphetamine,amphetamines,CHEMICALS,Effects,https://www.erowid.org/chemicals/amphetamines/...
4,Amphetamine,amphetamines,CHEMICALS,Chemistry,https://www.erowid.org/chemicals/amphetamines/...
...,...,...,...,...,...
124,Temazepam,temazepam,PHARMS,MainPage,https://www.erowid.org/pharms/temazepam/temaze...
125,Temazepam,temazepam,PHARMS,Law,https://www.erowid.org/pharms/temazepam/temaze...
126,Zolpidem,zolpidem,PHARMS,MainPage,https://www.erowid.org/pharms/zolpidem/zolpide...
127,Zolpidem,zolpidem,PHARMS,Law,https://www.erowid.org/pharms/zolpidem/zolpide...


In [10]:
# all unique URLs
url_cols = [c for c in long_matching_erowid_data.columns
            if c not in ("drug", "Substance", "Category")]

unique_urls = (
    pd.unique(
        long_matching_erowid_data[url_cols]
        .values                       # ndarray of all url‑cells
        .ravel("K")                   # flatten to 1‑D
    )
)

# drop NaNs / empty strings and convert to a clean Python list
unique_urls = [u for u in unique_urls if isinstance(u, str) and u.strip()]

print(f"{len(unique_urls)} unique URLs")
unique_urls   

170 unique URLs


['amphetamines',
 'buprenorphine',
 'codeine',
 'cyclobenzaprine',
 'diphenhydramine',
 'fentanyl',
 'gabapentin',
 'hydrocodone',
 'hydromorphone',
 'ketamine',
 'lorazepam',
 'lsd',
 'mda',
 'mdma',
 'meperidine',
 'methadone',
 'meth',
 'morphine',
 'naltrexone',
 'oxycodone',
 'oxymorphone',
 'pregabalin',
 'psilocybin',
 'temazepam',
 'zolpidem',
 'zopiclone',
 'CHEMICALS',
 'PHARMS',
 'MainPage',
 'Basics',
 'Law',
 'Effects',
 'Chemistry',
 'Testing',
 'Health',
 'History',
 'Journals',
 'Media',
 'FAQ',
 'Dose',
 'Writings',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines.shtml',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines_basics.shtml',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines_law.shtml',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines_effects.shtml',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines_chemistry.shtml',
 'https://www.erowid.org/chemicals/amphetamines/amphetamines_testing.shtml',
 'https://